In [1]:
# https://freeprog.tistory.com/329

import sys
import IPython
import imutils
import cv2
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *

from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import numpy as np
import argparse

import xlwings as xw
import os
import subprocess

QApplication.setAttribute(Qt.AA_EnableHighDpiScaling, True)



class Left(QWidget): 
    w= 0
    h= 0
    pixelsPerMetric=0
    fname= ''
    finame = ''
    sflist = []
    dir1= ''
    num = 0
    i= 0
    dB0=0
    exnum = 0
    last_directory=''
    width = 10
    def __init__(self):
    
        super().__init__()
           
        
        
        # 전체 폼 박스
        self.formbox = QHBoxLayout()
        self.setLayout(self.formbox)

        # 좌, 우 레이아웃박스
        self.left = QVBoxLayout()
        self.middle = QVBoxLayout()

        # 그룹박스1 생성 및 좌 레이아웃 배치
        gb = QGroupBox('그리기 종류')        
        self.left.addWidget(gb)

        # 그룹박스1 에서 사용할 레이아웃
        box = QVBoxLayout()
        gb.setLayout(box)        

        # 그룹박스 1 의 라디오 버튼 배치
        text = ['horizon']
        self.radiobtns = []

        for i in range(len(text)):
            self.radiobtns.append(QRadioButton(text[i], self))
            self.radiobtns[i].clicked.connect(self.radioClicked)
            box.addWidget(self.radiobtns[i])

        self.radiobtns[0].setChecked(True)
        self.drawType = 0
         
        # 그룹박스2
        gb = QGroupBox('펜 설정')        
        self.left.addWidget(gb)        

        grid = QGridLayout()      
        gb.setLayout(grid)        

        label = QLabel('선굵기')
        grid.addWidget(label, 0, 0)

        self.combo = QComboBox()
        grid.addWidget(self.combo, 0, 1)       

        for i in range(1, 21):
            self.combo.addItem(str(i))

        label = QLabel('선색상')

        grid.addWidget(label, 1,0)        
         
        self.pencolor = QColor(0,0,0)
        self.penbtn = QPushButton()        
        self.penbtn.setStyleSheet('background-color: rgb(0,0,0)')
        self.penbtn.clicked.connect(self.showColorDlg)
        grid.addWidget(self.penbtn,1, 1)
         

        # 그룹박스3
        gb = QGroupBox('붓 설정')        
        self.left.addWidget(gb)
        hbox = QHBoxLayout()
        gb.setLayout(hbox)

        label = QLabel('붓색상')
        hbox.addWidget(label)                

        self.brushcolor = QColor(255,255,255)
        self.brushbtn = QPushButton()        
        self.brushbtn.setStyleSheet('background-color: rgb(255,255,255)')
        self.brushbtn.clicked.connect(self.showColorDlg)
        hbox.addWidget(self.brushbtn)
        
        
        # 그룹박스4
        gb = QGroupBox('불러오기')        
        self.left.addWidget(gb)
        hbox = QHBoxLayout()
        gb.setLayout(hbox)    
            
        self.openbtn = QPushButton("File Open")
        self.openbtn.clicked.connect(self.openbtnClicked)
        self.label = QLabel()

        hbox.addWidget(self.openbtn)
        hbox.addWidget(self.label)
        
         # 그룹박스5
        gb = QGroupBox('불러오기 방법')        
        self.left.addWidget(gb)

        box2 = QHBoxLayout()
        gb.setLayout(box2)        

        text = ['*.확장자(unstack)', '개별(unstack)']
        self.radiobtns2 = []

        for i in range(len(text)):
            self.radiobtns2.append(QRadioButton(text[i], self))
            self.radiobtns2[i].clicked.connect(self.radio2Clicked)
            box2.addWidget(self.radiobtns2[i])

        self.radiobtns2[0].setChecked(True)
        self.loadType = 0
         
    
        #그룹박스 6
        gb=QGroupBox('처리할 목록')
        self.left.addWidget(gb)
        
        grid2= QGridLayout()
        gb.setLayout(grid2)
        
        
        self.tb = QTextBrowser()
        self.tb.setAcceptRichText(True)
        self.tb.setOpenExternalLinks(True)

        self.next_btn = QPushButton('Nextr')
        self.next_btn.pressed.connect(self.next_view)
        self.clear_btn = QPushButton('Clear')
        self.clear_btn.pressed.connect(self.clear_text)

        
        grid2.addWidget(self.tb, 0,0)
        grid2.addWidget(self.next_btn, 1,0)
        grid2.addWidget(self.clear_btn, 2,0)
        
        # 그룹박스7
        gb = QGroupBox('스케일 바')        
        
        self.left.addWidget(gb)
        grid3=QGridLayout()
        gb.setLayout(grid3)
        # LineEdit
        self.lineEdit = QLineEdit("", self)
#         self.lineEdit.returnPressed.connect(self.lineEditChanged)
        self.lineEdit.textChanged.connect(self.lineEditChanged)
        grid3.addWidget(self.lineEdit,0,0)
        self.statusBar = QStatusBar(self)
        
#         self.lbl_bar=QLabel("  ")
        grid3.addWidget(self.statusBar,0,1)
        
        
        
        # 그룹박스7
        gb = QGroupBox('스케일')        
        self.left.addWidget(gb)
        hbox = QHBoxLayout()
        gb.setLayout(hbox)
        
        self.scalebtn = QPushButton()  
        self.scalebtn.clicked.connect(self.scalepnt)
        
        hbox.addWidget(self.scalebtn)
        self.scalebtn.setText('print')
        
        self.lbl_1 = QLabel("  ")
        hbox.addWidget(self.lbl_1)
        
        
        # 그룹박스8
        gb = QGroupBox('크기 측정')        
        self.left.addWidget(gb)
        hbox = QHBoxLayout()
        gb.setLayout(hbox)
        
        self.pixelpntbtn = QPushButton()  
        self.pixelpntbtn.clicked.connect(self.pixelpnt)
        
        hbox.addWidget(self.pixelpntbtn)
        self.pixelpntbtn.setText('print')
        
        self.lbl_2 = QLabel("  ")
        hbox.addWidget(self.lbl_2)
            
         # 그룹박스9
        gb = QGroupBox('엑셀')        
        self.left.addWidget(gb)
        hbox = QHBoxLayout()
        gb.setLayout(hbox)
        
        self.excelbtn = QPushButton()  
        self.excelbtn.clicked.connect(self.excelup)
        
        hbox.addWidget(self.excelbtn)
        self.excelbtn.setText('엑셀업댓')
        
        self.lbl_3 = QLabel("  ")
        hbox.addWidget(self.lbl_3)
        
        self.left.addStretch(1)
        
        
        
        # 전체 폼박스에 왼쪽 박스 배치
        self.formbox.addLayout(self.left)
        self.formbox.setStretchFactor(self.left, 0)

        self.openbtnClicked()#############파일 선택
        
    def radioClicked(self):
        for i in range(len(self.radiobtns)):
            if self.radiobtns[i].isChecked():
                self.drawType = i                
                break

    def checkClicked(self):
        pass
             
    def showColorDlg(self):       
         
        # 색상 대화상자 생성      
        color = QColorDialog.getColor()

        sender = self.sender()

        # 색상이 유효한 값이면 참, QFrame에 색 적용
        if sender == self.penbtn and color.isValid():           
            self.pencolor = color
            self.penbtn.setStyleSheet('background-color: {}'.format( color.name()))
        else:
            self.brushcolor = color
            self.brushbtn.setStyleSheet('background-color: {}'.format( color.name()))

            

    def clear_text(self) :

        self.tb.clear()

        
    def radio2Clicked(self):
        for i in range(len(self.radiobtns2)):
            if self.radiobtns2[i].isChecked():
                self.loadType = i                
                break
        
    
    def openbtnClicked(self):
        
        
        self.tuplefname = QFileDialog.getOpenFileName(self)
        self.fname = self.tuplefname[0]
        print(self.fname)
        dir,file = os.path.split(self.fname)
        print("1 ",dir, "에 있는 ", file)
#         상대경로로 쓰려고 할때 작업공간 계속 바꿔줬는데, 절대경로로 불러오면 안씀.
#   같은 위치 디렉토리 설정 에러 피하기 위해 동일한 디렉토리로는 변경하지 않음.
        self.dir1=dir
        if self.dir1==Left.dir1:
            None
        else:
            os.chdir(dir)
        Left.dir1=self.dir1
        
        print("현재 작업공간은 ",os.getcwd())
        self.label.setText(file)
        self.finame = file
        self.foname =os.path.basename(dir)
        self.flist = os.listdir(dir)
        print("2 ", self.flist)
        print("3 ", os.getcwd())
        
        #선택: 클릭한 확장자의 파일명을 클릭한 파일이 있는 폴더 내에서 모두 찾아 리스트 저장하기.(초기화 후)
        #선택: 클릭한 파일만 리스트에 저장하기(초기화 후)
        #텍스트브라우저에 나타내기
        search = os.path.splitext(file)[1]
        print("4 확장자는 ",search)
        
        self.sflist =[]
        self.tb.clear()
        if self.loadType == 0:
            
            
            for name in self.flist:
                if search in name:
                    self.sflist.append(name)
                    print("5 처리할 데이터 목록은 ", self.sflist)
                    self.tb.append(name)
                else:
                    None
            Left.sflist= self.sflist
                    
        else:
            self.sflist.append(file)
            print("5 처리할 데이터는 ", self.sflist)
            self.tb.append(file)
            
        Left.sflist = self.sflist
        Left.fname = self.fname
        Left.finame = self.finame
        #원래의 Graview내용
        print("Graview안에 있는 w,h는 ", self.w, self.h)
        Left.i = 0
        Left.changeView(self)
        
    def changeView(self):
        if Left.num ==0:
            None
        else:
            Left.deleteItems(self.middle)
#             self.view.deleteLater()
#             self.view = None
        
        self.view =CView(self)
        
        (Left.w,Left.h,Left.pixelsPerMetric,Left.width,Left.dB0)=ppp.ppp(Left.sflist[Left.i])
        
        Left.h = self.h
        Left.w = self.w
        
        
        self.view.setFixedHeight(self.h)
        self.view.setFixedWidth(self.w)
        self.middle.addWidget(self.view)
         # 전체 폼박스에 오른쪽 박스 배치
        self.formbox.addLayout(self.middle)

        self.formbox.setStretchFactor(self.middle, 0)
        print("이번에 띄울 이미지는 ",Left.i,"번째 이미지")
        self.setWindowTitle(self.sflist[Left.i])
        self.setGeometry(100,30,self.w+200,self.h)

        Left.scalepnt(self)
        self.lbl_2.setText(None)
        Left.num +=1
        
        
    
    def deleteItems(layout): 
        if layout is not None: 
            while layout.count(): 
                item = layout.takeAt(0)
                print(item)
                widget = item.widget() 

                if widget is not None: 
                    widget.deleteLater() 
                else: 
                    deleteItems(item.layout()) 

                    
    def next_view(self):
        if Left.i ==len(self.sflist)-1:
            Left.i=0
        else: 
            Left.i+=1
        Left.changeView(self)
                    
                    
                
    def next_view(self):
        if Left.i ==len(self.sflist)-1:
            Left.i=0
        else: 
            Left.i+=1
        Left.changeView(self)
    
    
    def lineEditChanged(self):
        Left.width=int(self.lineEdit.text())
        self.statusBar.showMessage(self.lineEdit.text()+" "+chr(956)+"m") 
    
    
    
    def scalepnt(self):
        global dy
#         global pixelsPerMetric
        
        
        self.txt_1=repr("{} ".format(Left.width)+chr(956)+"m")+"길이당 {:}".format(Left.dB0)+" pixels"    
        
        
        self.lbl_1.setText(self.txt_1)
        self.lbl_1.adjustSize()   
            
    def pixelpnt(self):
        global dy
#         global pixelsPerMetric
        Left.pixelsPerMetric = Left.dB0 / Left.width
        print(Left.pixelsPerMetric)
        
        self.size= dy/Left.pixelsPerMetric
        
        self.txt_2="{}픽셀, ".format(dy)+repr("{:.5f} ".format(self.size)+chr(956)+"m")     
        
        
        self.lbl_2.setText(self.txt_2)
        self.lbl_2.adjustSize()    


    def excelup(self):
        #https://docs.xlwings.org/en/stable/quickstart.html
        folder=self.foname+"_xlsx_Pitdepth_data"
        current_directory = os.getcwd()
        if current_directory == Left.last_directory:
            wb = xw.Book(current_directory + os.sep+  folder + os.sep + "pit_depth_result.xlsx")
        else:
            wb = xw.Book()
        Left.last_directory = current_directory
        #이름이 있는 시트를 생성
        
        #(self.sflist[i],Left.width,self.size,self.foname)
        #Sheet1에다 입력
        sht1 = wb.sheets[0]
        sht1.range('A1').value = ['Data_name', 'scalebar ('+chr(956)+"m)", 'Pit_depth ('+chr(956)+"m)"]
        
        self.exnum=Left.exnum
        sht1.range('A'+str(2+self.exnum)).value = [self.sflist[Left.i], Left.width, self.size]
        Left.exnum=self.exnum+1


        #저장할 디렉토리를 알기위해, 현재의 디렉토리를 가져온다
        
        #디렉터리의 존재 여부를 확인하고, 없을 경우 해당 디렉터리를 생성
        
        try:
            if not(os.path.isdir(current_directory + os.sep+  folder)):
                os.makedirs(os.path.join(folder))
        except OSError as e:
            if e.errno != errno.EEXIST:
                print("Failed to create directory!!!!!")
                raise
        #현재디렉토리 아래의 xlsx_data라는 폴더에 작업한 엑셀파일을 저장
        #os.sep는 리눅스, 유니스 계열인 경우 슬래쉬, 윈도우 에서는 역슬래쉬 로 표시됨
        wb.save(current_directory + os.sep+  folder + os.sep + "pit_depth_result.xlsx")
#         time.sleep(0.5)
#         wb.close()


# QGraphicsView display QGraphicsScene
class CView(QGraphicsView):
    pics =[]
    num = 0
    def __init__(self, parent):
        
        super().__init__(parent)   
        
        self.dat= [Left.w,Left.h]
        print("CView로 받은 w,h 는 ",self.dat[:])
        
        print("CView에서의 작업공간은 ",os.getcwd())
        print("파일명은",Left.sflist[Left.i])
        
        
        self.scene = QGraphicsScene()
        

        pic = QPixmap('./'+Left.sflist[Left.i])

        self.scene.addPixmap(pic)
        self.setScene(self.scene)
        

        
        self.items = []
        
        self.start = QPointF()
        self.end = QPointF()
        self.setRenderHint(QPainter.HighQualityAntialiasing)  
        
#     @staticmethod    
#     def getwhfname(w,h,fname):
        
#         return w, h, fname
        
    def moveEvent(self, e): # 없애면 스크롤바 생김
        rect = QRectF(self.rect())
        rect.adjust(0,0,-2,-2)

        self.scene.setSceneRect(rect)
    
    def mousePressEvent(self, e):
        
        if e.buttons() & Qt.RightButton:           
            if len(self.items) > 0:
                self.scene.removeItem(self.items[-1])
                del(self.items[-1])

        if e.button() == Qt.LeftButton:
            # 시작점 저장
            self.start = e.pos()
            self.end = e.pos() 
            
    
    def mouseMoveEvent(self, e):  
        
        if e.buttons() & Qt.RightButton:           
            if len(self.items) > 0:
                self.scene.removeItem(self.items[-1])
                del(self.items[-1])
                
        # e.buttons()는 정수형 값을 리턴, e.button()은 move시 Qt.Nobutton 리턴 
        if e.buttons() & Qt.LeftButton:           

            self.end = e.pos()

            pen = QPen(self.parent().pencolor, self.parent().combo.currentIndex())

        
            # 수평선 그리기
            if self.parent().drawType == 0:
                 
                # 장면에 그려진 이전 선을 제거            
                if len(self.items) > 0:
                    self.scene.removeItem(self.items[-1])
                    del(self.items[-1])  
                    self.scene.removeItem(self.items[-1])
                    del(self.items[-1])  
                
                hori1 = QLineF(0, self.start.y(), Left.w, self.start.y())
                
                self.items.append(self.scene.addLine(hori1, pen))
                
                hori2 = QLineF(self.end.x(), self.start.y(), self.end.x(), self.end.y())
                
                self.items.append(self.scene.addLine(hori2, pen))
                print(self.start.y(),self.end.y())
                
                
                
                """
                # 현재 선 추가
                hori = QLineF(self.start.x(), self.start.y(), self.end.x(), self.start.y())                
                self.items.append(self.scene.addLine(hori, pen))
                """

                
                
                
                
            
    
    def mouseReleaseEvent(self, e):  
        
        if e.button() == Qt.RightButton:
            if len(self.items) > 0:
                self.scene.removeItem(self.items[-1])
                #del(self.items[-1])    

        if e.button() == Qt.LeftButton:

            pen = QPen(self.parent().pencolor, self.parent().combo.currentIndex())
            global dy 
            dy = abs(self.start.y()-self.end.y())
            self.dx = abs(self.start.x()-self.end.x())
            print(dy, self.dx)
            

    """
            if self.parent().drawType == 0:
                
                self.items.clear()
                hori = QLineF(0, self.start.y(), w, self.start.y())
                self.scene.addLine(hori, pen)
                hori = QLineF(self.end.x(), self.start.y(), self.end.x(), self.end.y())
                self.scene.addLine(hori, pen)
                print(self.start,self.end)
    """  
    
    
class ppp():
    
    @staticmethod
    def midpoint(ptA, ptB):
        return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)  
    @staticmethod
    def ppp(fname):
    
       
        # global fname
    #     fname = './images/example_04.bmp'
        width=Left.width





        # load the image, convert it to grayscale, and blur it slightly
        image =cv2.imread(fname)
#         cv2.imshow('image',image)
        h, w = image.shape[:2]
        cv2.waitKey(0)
        h1=round(0.935*h)
        h2=round(0.97*h)
#         w2=round(0.691*w)
        w2=round(0.65*w)
        print(h1, h2, w2)
        image = image[h1:h2,0:w2]


        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        blur = cv2.GaussianBlur(gray, (7, 7), 0)

        # perform edge detection, then perform a dilation + erosion to
        # close gaps in between object edges
        canny = cv2.Canny(blur, 50, 100)
        edged = cv2.dilate(canny, None, iterations=1)
        edged = cv2.erode(edged, None, iterations=1)

        # find contours in the edge map
        cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        
        cnts = imutils.grab_contours(cnts)

        # sort the contours from left-to-right and initialize the
        # 'pixels per metric' calibration variable
        (cnts, _) = contours.sort_contours(cnts)
    #     global pixelsPerMetric
        pixelsPerMetric = None

        # loop over the contours individually
        for c in cnts:
            # if the contour is not sufficiently large, ignore it
            if cv2.contourArea(c) < 100:
                continue

            # compute the rotated bounding box of the contour
            orig = image.copy()
            box = cv2.minAreaRect(c)
            box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
            box = np.array(box, dtype="int")
            #print(box)

            # order the points in the contour such that they appear
            # in top-left, top-right, bottom-right, and bottom-left
            # order, then draw the outline of the rotated bounding
            # box
            box = perspective.order_points(box)
            cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)

            # loop over the original points and draw them
            for (x, y) in box:
                cv2.circle(orig, (int(x), int(y)), 5, (0, 0, 255), -1)

            # unpack the ordered bounding box, then compute the midpoint
            # between the top-left and top-right coordinates, followed by
            # the midpoint between bottom-left and bottom-right coordinates
            (tl, tr, br, bl) = box
            (tltrX, tltrY) = ppp.midpoint(tl, tr)
            (blbrX, blbrY) = ppp.midpoint(bl, br)

            # compute the midpoint between the top-left and top-right points,
            # followed by the midpoint between the top-righ and bottom-right
            (tlblX, tlblY) = ppp.midpoint(tl, bl)
            (trbrX, trbrY) = ppp.midpoint(tr, br)

            # draw the midpoints on the image
            cv2.circle(orig, (int(tltrX), int(tltrY)), 5, (255, 0, 0), -1)
            cv2.circle(orig, (int(blbrX), int(blbrY)), 5, (255, 0, 0), -1)
            cv2.circle(orig, (int(tlblX), int(tlblY)), 5, (255, 0, 0), -1)
            cv2.circle(orig, (int(trbrX), int(trbrY)), 5, (255, 0, 0), -1)

            # draw lines between the midpoints
            cv2.line(orig, (int(tltrX), int(tltrY)), (int(blbrX), int(blbrY)),
                (255, 0, 255), 2)
            cv2.line(orig, (int(tlblX), int(tlblY)), (int(trbrX), int(trbrY)),
                (255, 0, 255), 2)

            # compute the Euclidean distance between the midpoints
            dA = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))
            dB = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))

            
            if pixelsPerMetric is None:
                pixelsPerMetric = dB / width
                dB0=dB
            # compute the size of the object
            dimA = dA / pixelsPerMetric
            dimB = dB / pixelsPerMetric

            # draw the object sizes on the image
            cv2.putText(orig, "{:.5f}".format(dimB),
                (int(tltrX - 15), int(tltrY - 10)), cv2.FONT_HERSHEY_SIMPLEX,0.65, (255, 255, 255), 2)
            cv2.putText(orig, "{:.5f}".format(dimA),(int(trbrX + 10), int(trbrY)),
                cv2.FONT_HERSHEY_SIMPLEX,0.65, (255, 255, 255), 2)   


            
        return w,h, pixelsPerMetric, width,dB0




if __name__=="__main__":
    app = QApplication(sys.argv)
    ex = Left()
    ex.show()
    sys.exit(app.exec_())


//hj/Thermal/2019 부식/19년도 부식실험 - FE-SEM 이미지/190605 - 3000H Shot-peening/새 폴더/8-17 down_15kV PC6 WD10_22.bmp
1  //hj/Thermal/2019 부식/19년도 부식실험 - FE-SEM 이미지/190605 - 3000H Shot-peening/새 폴더 에 있는  8-17 down_15kV PC6 WD10_22.bmp
현재 작업공간은  \\hj\Thermal\2019 부식\19년도 부식실험 - FE-SEM 이미지\190605 - 3000H Shot-peening\새 폴더
2  ['8-17 down_15kV PC6 WD10_22.bmp', '8-17 down_15kV PC6 WD10_22.txt', '8-17 down_15kV PC6 WD10_23.bmp', '8-17 down_15kV PC6 WD10_23.txt', '8-17 down_15kV PC6 WD10_24.bmp', '8-17 down_15kV PC6 WD10_24.txt', '8-17 down_15kV PC6 WD10_25.bmp', '8-17 down_15kV PC6 WD10_25.txt', '8-17 down_15kV PC6 WD10_26.bmp', '8-17 down_15kV PC6 WD10_26.txt', '8-17 down_15kV PC6 WD10_27.bmp', '8-17 down_15kV PC6 WD10_27.txt', '8-17 down_15kV PC6 WD10_28.bmp', '8-17 down_15kV PC6 WD10_28.txt', '8-17 down_15kV PC6 WD10_29.bmp', '8-17 down_15kV PC6 WD10_29.txt', '8-17 down_15kV PC6 WD10_30.bmp', '8-17 down_15kV PC6 WD10_30.txt', '8-17 down_15kV PC6 WD10_31.bmp', '8-17 down_15kV PC6 WD10_31.txt', '8-1

CView로 받은 w,h 는  [1280, 1024]
CView에서의 작업공간은  \\hj\Thermal\2019 부식\19년도 부식실험 - FE-SEM 이미지\190605 - 3000H Shot-peening\새 폴더
파일명은 8-17 down_15kV PC6 WD10_23.bmp
957 993 832
이번에 띄울 이미지는  1 번째 이미지
248 248
248 250
248 254
248 259
248 268
248 281
248 295
248 310
248 324
248 338
248 349
248 359
248 364
248 369
248 373
248 375
248 376
248 376
248 377
248 378
248 378
248 378
248 379
248 381
248 381
248 384
248 387
248 389
248 391
248 393
248 395
248 398
248 399
248 400
248 402
248 403
248 404
248 405
248 406
248 407
248 408
248 409
248 409
248 409
248 409
248 409
248 410
248 411
248 411
248 411
248 411
248 411
248 411
248 412
248 412
248 413
248 414
248 414
248 414
248 414
248 415
248 415
248 416
248 416
248 416
248 416
248 417
248 417
248 417
248 418
248 419
248 419
248 420
248 421
248 421
248 421
248 422
248 423
248 423
248 424
248 424
248 424
248 424
248 424
248 424
248 424
248 424
248 424
248 424
248 424
248 424
248 424
248 424
248 424
248 424
248 424
248 423
248 421
248 421
248 420
248 419

411 514
411 515
411 515
411 515
411 516
411 517
411 517
411 517
411 517
411 516
411 515
411 514
411 513
411 513
411 512
411 511
411 510
99 261
3.3
CView로 받은 w,h 는  [1280, 1024]
CView에서의 작업공간은  \\hj\Thermal\2019 부식\19년도 부식실험 - FE-SEM 이미지\190605 - 3000H Shot-peening\새 폴더
파일명은 8-17 down_15kV PC6 WD10_30.bmp
957 993 832
이번에 띄울 이미지는  8 번째 이미지
469 469
469 469
469 469
469 469
469 469
469 469
469 470
469 473
469 478
469 484
469 490
469 496
469 508
469 522
469 535
469 549
469 554
469 560
469 565
469 569
469 571
469 572
469 573
469 574
469 575
469 575
469 575
469 575
469 575
469 575
469 576
469 576
469 576
469 576
469 576
469 575
469 573
469 572
469 571
469 569
469 568
469 566
469 564
469 563
469 561
469 560
469 559
469 558
469 558
469 557
469 557
469 557
469 556
469 556
469 556
469 555
469 555
469 555
469 555
469 555
469 554
469 553
469 552
469 552
469 551
469 550
469 549
469 549
469 549
469 548
469 548
469 548
469 545
469 544
469 544
469 543
469 542
469 541
469 540
469 540
469 540
469 539
469 

376 456
376 458
376 459
376 461
376 463
376 466
376 469
376 472
376 475
376 478
376 481
376 483
376 485
376 487
376 488
376 490
376 490
376 492
376 493
376 495
376 498
376 500
376 502
376 505
376 507
376 508
376 509
376 511
376 512
376 512
376 514
376 515
376 516
376 515
376 514
376 514
376 513
137 737
5.4
CView로 받은 w,h 는  [1280, 1024]
CView에서의 작업공간은  \\hj\Thermal\2019 부식\19년도 부식실험 - FE-SEM 이미지\190605 - 3000H Shot-peening\새 폴더
파일명은 8-17 down_15kV PC6 WD10_37.bmp
957 993 832
이번에 띄울 이미지는  15 번째 이미지
343 344
343 347
343 351
343 356
343 361
343 368
343 375
343 383
343 392
343 398
343 406
343 411
343 415
343 418
343 421
343 422
343 423
343 424
343 425
343 426
343 427
343 428
343 429
343 430
343 431
343 433
343 433
343 435
343 437
343 440
343 441
343 443
343 446
343 448
343 449
343 450
343 452
343 453
343 454
343 456
343 457
343 458
343 459
343 460
343 461
343 463
343 465
343 467
343 469
343 472
343 474
343 476
343 478
343 481
343 483
343 485
343 487
343 488
343 489
343 490
343 491
343 492
34

283 285
283 289
283 293
283 297
283 303
283 309
283 316
283 325
283 336
283 347
283 360
283 370
283 376
283 382
283 387
283 390
283 392
283 395
283 397
283 400
283 403
283 408
283 414
283 419
283 427
283 432
283 436
283 441
283 444
283 448
283 451
283 454
283 457
283 462
283 468
283 473
283 480
283 486
283 492
283 498
283 503
283 505
283 508
283 509
283 511
283 511
283 513
283 514
283 515
283 517
283 517
283 519
283 520
283 521
283 522
283 522
283 523
283 524
283 524
283 524
283 524
283 524
283 524
283 524
283 524
283 524
283 525
283 525
283 525
283 525
283 526
283 526
283 526
283 526
283 526
283 527
283 527
283 528
283 528
283 528
283 529
283 529
283 529
246 696
5.4
CView로 받은 w,h 는  [1280, 1024]
CView에서의 작업공간은  \\hj\Thermal\2019 부식\19년도 부식실험 - FE-SEM 이미지\190605 - 3000H Shot-peening\새 폴더
파일명은 8-17 down_15kV PC6 WD10_40.bmp
957 993 832
이번에 띄울 이미지는  18 번째 이미지
265 265
265 266
265 267
265 269
265 270
265 272
265 275
265 277
265 278
265 280
265 281
265 284
265 286
265 288
265 291
265 292
26

...

IndexError: list index out of range

294 295
294 296
294 296
294 298
294 298
294 298
294 300
294 301
294 301
294 302
294 302
294 304
294 304
294 304
294 305
294 306
294 306
294 306
294 308
294 308
294 308
294 308
294 309
294 312
294 316
294 320
294 323
294 326
294 327
294 327
294 327
294 327
294 327
294 327
294 327
294 328
294 328
294 328
294 328
294 328
294 331
294 332
294 335
294 339
294 340
294 343
294 347
294 349
294 352
294 353
294 355
294 356
294 357
294 358
294 359
294 361
294 361
294 361
294 361
294 361
294 361
294 362
294 362
294 363
294 365
294 365
294 367
294 369
294 371
294 374
294 376
294 379
294 382
294 385
294 387
294 389
294 389
294 389
294 389
294 390
294 390
294 391
294 391
294 391
294 391
294 391
294 391
294 391
294 391
294 391
294 391
294 391
294 391
294 391
294 391
294 391
294 391
294 391
294 391
294 391
294 391
294 391
294 391
294 391
294 391
294 391
294 391
294 391
294 392
294 392
294 392
294 392
294 393
294 393
294 393
294 393
294 393
294 393
294 393
294 393
294 393
294 393
294 393
294 392
294 391


447 447
447 447
447 448
447 448
447 449
447 450
447 451
447 451
447 452
447 453
447 454
447 457
447 458
447 460
447 461
447 463
447 465
447 465
447 466
447 466
447 467
447 467
447 468
447 468
447 468
447 469
447 469
447 470
447 471
447 472
447 473
447 474
447 474
447 475
447 476
447 476
447 478
447 478
447 480
447 482
447 484
447 487
447 489
447 490
447 492
447 493
447 494
447 495
447 496
447 497
447 498
447 498
447 498
447 498
447 498
447 499
447 500
447 500
447 501
447 503
447 504
447 505
447 507
447 508
447 509
447 510
447 511
447 513
447 514
447 515
447 516
447 517
447 518
447 519
447 520
73 172
5.4
CView로 받은 w,h 는  [1280, 1024]
CView에서의 작업공간은  \\hj\Thermal\2019 부식\19년도 부식실험 - FE-SEM 이미지\190605 - 3000H Shot-peening\새 폴더
파일명은 8-18 up_15kV PC6 WD10_68.bmp
957 993 832
이번에 띄울 이미지는  108 번째 이미지
CView로 받은 w,h 는  [1280, 1024]
CView에서의 작업공간은  \\hj\Thermal\2019 부식\19년도 부식실험 - FE-SEM 이미지\190605 - 3000H Shot-peening\새 폴더
파일명은 8-17 down_15kV PC6 WD10_22.bmp
957 993 832
이번에 띄울 이미지는  0 번째 이미지
CVi

SystemExit: 0

C:\Users\Hojin\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
